### Imports

In [10]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import pandas 

### Data Preparation

In [43]:
data = pandas.read_csv('./data/Learning/Training_set.csv')
train_set = data[['x','y','z']]
target = data['brakingStatus']

test_data = pandas.read_csv('./data/21/aceleracaoLinear_terra.csv')[['x','y','z']]

In [44]:
train_set = np.array(train_set,dtype=float)
target = np.array(target,dtype=float)

In [45]:
train_set.shape

(20675, 3)

In [46]:
target.shape

(20675,)

### RNN model

In [47]:
model = Sequential()

In [51]:
model.add(LSTM(1,batch_input_shape= (None,3,1),return_sequences= False))

Instructions for updating:
Colocations handled automatically by placer.


In [53]:
model.compile(loss='mean_absolute_error',optimizer='adam',metrics=['accuracy'])

In [54]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 1)                 12        
Total params: 12
Trainable params: 12
Non-trainable params: 0
_________________________________________________________________
